In [1]:
import os
import pandas as pd

In [7]:
global_north = ['Australia','England','New Zealand','South Africa']
global_south = ['India','Pakistan','Bangladesh','West Indies']

In [124]:

def doAnalysis(fileName,country,image_file_path,status_csv_file,API_NAME):
    df = pd.read_csv(fileName)
    df_status = pd.read_csv(status_csv_file+".csv")
    
    ##################################################################################################
    columns = df.columns[2:-2]
    df_res = pd.DataFrame(['Overall','Male','Female'])
    df_res_total = pd.DataFrame(['Overall','Male','Female'])
    test = pd.DataFrame(['Overall','Male','Female'])
    
#     print(columns)
    
    for column in columns:
        ################# Consider only valid images for analysis ##############
        df_res[column.title()] = None
        df_res_total[column.title()] = None
        test[column.title()] = None
        
        image_type = 'normal'
        if column == 'masked':
            image_type = 'masked'
        df_copy = df[['name','gender',column,'valid '+image_type]]
        df_copy=df_copy.rename(columns = {'valid '+image_type:'valid'})
        df_copy = df_copy[df_copy['valid'] == 1]
        
        ############ There are some greyscale and cartoonish images in the folder, we should  ############
        ##### not consider those images for our accuracy plot #####
        img_type = column.split()[0].lower()
        df_status['fileName'] = df_status['fileName'].apply(lambda x: x.split(".")[0])
        df_copy = df_copy.merge(df_status, left_on='name', right_on='fileName', how='inner')[['name','gender', column,'valid', 'status']]
        if img_type == "greyish":
            df_copy = df_copy[df_copy['status'].isin(['OK', 'GREY'])]
        else:
            df_copy = df_copy[df_copy['status'] == 'OK']
            
#         print(df_copy)
        ###################################################################
        
#         column = column.capitalize()
        total = len(df_copy)
        df_male = df_copy.copy()
        df_male = df_male[df_male['gender'] == 'M']
        df_female = df_copy.copy()
        df_female = df_female[df_female['gender'] == 'F']
        
        total_male = len(df_male)
        total_female = len(df_female)
        
        total_correct_prediction = len(df_copy[df_copy['gender'] == df_copy[column]])
        correct_prediction_male = len(df_male[df_male['gender'] == df_male[column]])
        correct_prediction_female = len(df_female[df_female['gender'] == df_female[column]])
        
        overall_accuracy = (total_correct_prediction/total)*100
        male_accuracy = (correct_prediction_male/total_male)*100
        female_accuracy = (correct_prediction_female/total_female)*100
        
        df_res[column.title()][0] = total_correct_prediction
        df_res[column.title()][1] = correct_prediction_male
        df_res[column.title()][2] = correct_prediction_female
        
        df_res_total[column.title()][0] = total
        df_res_total[column.title()][1] = total_male
        df_res_total[column.title()][2] = total_female
        
        test[column.title()][0] = round(overall_accuracy,2)
        test[column.title()][1] = round(male_accuracy,2)
        test[column.title()][2] = round(female_accuracy,2)
        
        
    if API_NAME == "aws":
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
        df_res_total=df_res_total.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
        test = test.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
    elif API_NAME == 'azure':
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
        df_res_total=df_res_total.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
        test = test.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
    elif API_NAME == 'facepp':
        df_res=df_res.rename(columns = {'Original Normal':'ORIGINAL','Original Masked':'MASKED','Greyish Normal':'GREYSCALE',
                       'Rgb0.3 Normal':'RGB-0.3','Rgb0.5 Normal':'RGB-0.5','Spread Normal':'SPREAD'})
        df_res_total=df_res_total.rename(columns = {'Original Normal':'ORIGINAL','Original Masked':'MASKED','Greyish Normal':'GREYSCALE',
                       'Rgb0.3 Normal':'RGB-0.3','Rgb0.5 Normal':'RGB-0.5','Spread Normal':'SPREAD'})
        
        test = test.rename(columns = {'Original Normal':'ORIGINAL','Original Masked':'MASKED','Greyish Normal':'GREYSCALE',
                       'Rgb0.3 Normal':'RGB-0.3','Rgb0.5 Normal':'RGB-0.5','Spread Normal':'SPREAD'})
    elif API_NAME == "deepface_without_face_detection" or API_NAME == "deepface":
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3 Noise':'RGB-0.3','Rgb0.5 Noise':'RGB-0.5','Spread Noise':'SPREAD'})
        df_res_total=df_res_total.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3 Noise':'RGB-0.3','Rgb0.5 Noise':'RGB-0.5','Spread Noise':'SPREAD'})
        
        test = test.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3 Noise':'RGB-0.3','Rgb0.5 Noise':'RGB-0.5','Spread Noise':'SPREAD'})
        
    elif API_NAME == 'libfaceid':
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
        df_res_total=df_res_total.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
        
        test = test.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                       'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})

    index = [0,'ORIGINAL','GREYSCALE','RGB-0.3','RGB-0.5','SPREAD','MASKED']
    df_res = df_res[index]
    df_res_total = df_res_total[index]
    test = test[index]
    
#         print(column.title())
    
#     print("########### ",country," ###########")
#     print(test)
#     print(df_res)
#     print(df_res_total)
    return df_res,df_res_total

In [131]:
API_NAME = "libfaceid"
for_gn = True

path = "./../Results in csv files/"+API_NAME+'/'
columns = ['country','ORIGINAL','GREYSCALE','RGB-0.3','RGB-0.5','SPREAD','MASKED']


image_file_path = "./../All Face Images with And Without Mask (N95)/"
status_csv_file = "./../correction/status_csv_files/"


countries = os.listdir(path)

df_res = None
df_res_total = None

for for_gn in [True,False]:
    first_time = True
    for country in countries:
        country = country.split('.')[0]
        if for_gn == True:
            if country not in global_north:
                continue
        else:
            if country not in global_south:
                continue

        filePath = "./../Results in csv files/"+API_NAME+"/"+country+'.csv'
        df_corr, df_total = doAnalysis(filePath,country,image_file_path,status_csv_file+country,API_NAME)

        if first_time:
            df_res = df_corr.copy()
            df_res_total = df_total.copy()
            first_time = False
        else:
            df_res_temp = df_res.copy()
            df_res_total_temp = df_res_total.copy()

            df_res.iloc[:, 1:] = df_res_temp.iloc[:, 1:] + df_corr.iloc[:, 1:]
            df_res_total.iloc[:, 1:] = df_res_total_temp.iloc[:, 1:] + df_total.iloc[:, 1:]
    # print()   
    result_df = df_res.copy()
    result_df.iloc[:, 1:] = (df_res.iloc[:, 1:] / df_res_total.iloc[:, 1:]) * 100

    result_df.iloc[:, 1:] = result_df.iloc[:, 1:].applymap(lambda x: round(x, 2))

    file_name = "global_north" if for_gn else "global_south"
    print(file_name)
    print(result_df)
    result_df.to_csv(path+file_name+'.csv',index = False)
    print()

global_north
         0 ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0  Overall    84.26     82.05   81.23   76.61  83.71  81.37
1     Male    97.02     94.31   84.95   78.44  94.45  99.39
2   Female    30.25     23.64   65.48   68.86  38.26   5.66

global_south
         0 ORIGINAL GREYSCALE RGB-0.3 RGB-0.5 SPREAD MASKED
0  Overall    90.08      88.5   86.37   81.89  90.08  89.37
1     Male    98.53      96.3   91.28   86.23  97.34  98.94
2   Female    15.18      15.9   42.82   43.36  25.75   5.72



In [142]:
API_NAME = "deepface_without_face_detection"

path = "./../Results in csv files/"+API_NAME+'/'
files = os.listdir(path)
columns = ['region','ORIGINAL','GREYSCALE','RGB-0.3','RGB-0.5','SPREAD','MASKED']

df_ov = pd.DataFrame(columns = columns)
df_m = pd.DataFrame(columns = columns)
df_f = pd.DataFrame(columns = columns)

for file in files:
    if file.split('.')[0] == "global_north" or file.split('.')[0] == "global_south":
        df_acc = pd.read_csv(path+file)
        print(df_acc)
        region = file.split('.')[0]
        lis_ov = list(df_acc.iloc[0,:])[1:]
        lis_m = list(df_acc.iloc[1,:])[1:]
        lis_f = list(df_acc.iloc[2,:])[1:]
        
        df_ov.at[len(df_ov.index)] = [region]+lis_ov
        df_m.at[len(df_m.index)] = [region]+lis_m
        df_f.at[len(df_f.index)] = [region]+lis_f

df_ov.to_csv(path+"ns_overall.csv",index = False)
df_m.to_csv(path+"ns_male.csv",index = False)
df_f.to_csv(path+"ns_female.csv",index = False)

         0  ORIGINAL  GREYSCALE  RGB-0.3  RGB-0.5  SPREAD  MASKED
0  Overall     95.34      95.96    89.90    83.82   91.23   89.86
1     Male     99.92      99.97    99.92    99.96   99.87   99.87
2   Female     75.98      76.86    47.51    15.48   54.63   47.81
         0  ORIGINAL  GREYSCALE  RGB-0.3  RGB-0.5  SPREAD  MASKED
0  Overall     92.44      93.05    91.15    90.13   91.53   91.55
1     Male     99.91      99.91   100.00    99.97   99.88   99.94
2   Female     26.29      29.11    12.74     2.98   17.62   18.26
